In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Google A2A (Agent-to-Agent) Communication

This notebook introduces you to Google's Agent-to-Agent (A2A) protocol, a standardized way for AI agents to communicate and collaborate.  

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fa2aproject%2Fa2a-samples%2Fmain%2Fnotebooks%2Fa2a_quickstart.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/a2aproject/a2a-samples/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## What You'll Build

A three-agent system that works together to analyze trending topics:
1. **Trending Topics Agent** - Searches the web for current trending topics
2. **Trend Analyzer Agent** - Performs deep analysis with quantitative data
3. **Host Agent** - Orchestrates the other agents to provide comprehensive insights

<img src="https://storage.googleapis.com/github-repo/a2a/a2a-diagram.png" alt="drawing" width="1000"/>

## Prerequisites

- Python 3.11+
- Google Cloud Project with Vertex AI enabled
- Basic understanding of async Python

## Other Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai)
- Codelabs:
  - [Google's Agent Stack in Action: ADK, A2A, MCP on Google Cloud](https://codelabs.developers.google.com/instavibe-adk-multi-agents/instructions)
  - [Getting Started with Agent-to-Agent (A2A) Protocol: Gemini on Cloud Run](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
  - [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent)

#### Important!
A2A is a work in progress (WIP) thus, in the near future there might be changes that are different from what demonstrated in this code.

### Setup and Installation

First, let's install the required dependencies:

## 1. Introduction to A2A

### What is Agent-to-Agent (A2A) Communication?

A2A is a standardized protocol that enables AI agents to:
- **Discover** each other's capabilities
- **Communicate** using a common JSON-RPC based protocol
- **Collaborate** to solve complex tasks
- **Stream** responses for real-time interactions

### Environment Configuration

In [1]:
# Targeted workaround for google-adk==1.9.0 compatibility with a2a-sdk==0.3.0
# This cell shall be removed when google-adk releases the version next to >1.9.0
# (after https://github.com/google/adk-python/pull/2297)


import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [2]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn
import pandas as pd

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

In [3]:
load_dotenv()

azure_model = LiteLlm(
    model="gpt-5-mini", # gpt-4o-mini, gpt-5.1
    api_base=os.getenv('OPENAI_BASE_URL'),
    api_key=os.getenv('OPENAI_API_KEY'),
    api_version="2024-08-01-preview",
)

In [4]:
# Setup logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)

## 2. Building Your A2A System

Let's build our three-agent system step by step. We'll create:

1. **Trending Topics Agent** - Finds current trending topics
2. **Trend Analyzer Agent** - Analyzes trends with quantitative data
3. **Host Agent** - Orchestrates the other agents (sequentially)

### Agent 1: Trending Topics Agent

This agent searches the web for trending topics and returns a list of current trends.

In [5]:
def get_utbildningar() -> str:
    return pd.read_csv('mock_data/onboarding_kurser.csv').to_json()

hamta_utbildningar_agent = Agent(
    model=azure_model,
    name='hamta_utbildningar',
    instruction="""
    Du är en agent som baserat på medarbetares roll hämtar ut vilka utbildningar som den personen måste ta. Du skickar tillbaka en lista i json format med dessa utbildnigar.

    När du får en fråga om utbildningar:
    1. hitta utbildningar för den specifika rollen
    2. sammanställ en lista med kurs_id,kurs_namn,kategori,beskrivning,obligatorisk

    When asked about trends:
    1. Search for "trending topics today" or similar queries
    2. Extract the top 3 trending topics
    3. Return them in a JSON format

    Focus on current, real-time trends from the last 24 hours.

    You MUST return your response in the following JSON format, här är ett exempel:
    {
        "sjuksköterska": [
            {
                "kurs_id": "01",
                "kurs_namn": "Vuxen HLR",
                "kategori": "HLR",
                "beskrivning": "grundkurs där du lär du utföra HLR på vuxna personer",
                "obligatorisk": "ja"
            },
            {
                "kurs_id": "02",
                "kurs_namn": "släckövning",
                "kategori": "brandutbildning",
                "beskrivning": "kurs i hur man släcker en eld",
                "obligatorisk": "ja"
            }
        ]
    }

    Only return the JSON object, no additional text.

    Hittar du inget så returnerar du bara det exempel som vi gav dig i instruktionen.
    """,
    tools = [get_utbildningar]
)

print('hämta utbildningar Agent created successfully!')

hämta utbildningar Agent created successfully!


In [6]:
hamta_utbildningar_agent_card = AgentCard(
    name='Hämta utbildningar',
    url='http://localhost:10020',
    description='Hittar vilka utbildningar som en specifik medarbetargrupp ska ha tagit.',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='hamta_utbildning',
            name='hamta utbildning',
            description='Hittar vilka utbildningar som en specifik medarbetargrupp ska ha tagit.',
            tags=['utbildning'],
            examples=[
                "Vilka kurser ska en sjuksköterska gå?",
            ],
        )
    ],
)

In [7]:
remote_hamta_utbildningar = RemoteA2aAgent(
    name='hamta_utbildningar',
    description='Hittar vilka utbildningar som en specifik medarbetargrupp ska ha tagit.',
    agent_card=f'http://localhost:10020{AGENT_CARD_WELL_KNOWN_PATH}',
)

/tmp/ipykernel_23227/3634814921.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_hamta_utbildningar = RemoteA2aAgent(


### Agent 2: Trend Analyzer Agent

This agent takes a specific trend and performs deep analysis with quantitative data.

In [8]:
# Create the hämta utbildningar Agent
def get_schedule() -> str:
    return pd.read_csv('mock_data/kurs_schema.csv').to_json()

skapa_utbildning_agent = Agent(
    model=azure_model,
    name='skapa_utbildningsutbudsschema',
    instruction="""
    Du är en agent som har tillgång till schemat för alla utbildningar som erbjuds. Ditt jobb är att baserat på information om vad en spcifik medarbetargrupp behöver gå för utbildningar tar fram en data frame med vilka möjliga tillfällen som erbjuds.

    När du får en fråga om att skapa ett schema med relevanta utbildningar:
    1. hitta vilka utbildningar som personen ska gå
    2. ta fram en kalender med alla tillfällen som utbildningarna erbjuds
    4. välj ut de mest närliggande tiderna
    3. sätt ihop en data frame med de utbldningarna som personen obligatoriskt måste ta
    4. markera eventuellt överlappande kurstillfällen.

    I din dataframe ta med schema_id,kurs_id,datum,starttid,sluttid,plats,instruktör,max_deltagare

    Hittar du inget så returnerar du bara ett exempel som du hittar på.
    """,
    tools = [get_schedule]
)

print('hämta utbildningar Agent created successfully!')

hämta utbildningar Agent created successfully!


In [9]:
skapa_utbildning_agent_card = AgentCard(
    name='skapa utbildningsutbudsschema',
    url='http://localhost:10021',
    description='Skapar ett schema över vilka kursertillfällen som finns att välja på för de obligatoriska kurser som personen måste gå',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='hamta_utbildningsutbud',
            name='hämta utbildningsutbud',
            description='Skapar ett schema över vilka kursertillfällen som finns att välja på för de obligatoriska kurser som personen måste gå',
            tags = ['schema'],
            examples=[
                "när erbjuds vuxen HLR kursen?",
            ],
        )
    ],
)

In [10]:
remote_skapa_utbildningsschema = RemoteA2aAgent(
    name='hamta_utbildningsutbud',
    description='Skapar ett schema över vilka kursertillfällen som finns att välja på för de obligatoriska kurser som personen måste gå',
    agent_card=f'http://localhost:10021{AGENT_CARD_WELL_KNOWN_PATH}',
)

/tmp/ipykernel_23227/2883632773.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_skapa_utbildningsschema = RemoteA2aAgent(


Agent - Hämta personalschema

In [11]:
# Create the hämta personalschema Agent
def get_kliniskt_schema() -> str:
    return pd.read_csv('mock_data/personal_schema.csv').to_json()

hamta_personalschema_agent = Agent(
    model=azure_model,
    name='hamta_personalschema',
    instruction="""
    Du är en agent som har tillgång till schemat för all personal.
    Ditt jobb är att hitta när en specifik medarbetare arbetar.

    När du får en fråga om en medarbetares schema:
    1. hitta medaarbetarens schema
    2. Returnera schemat för medarbetaren

    I din dataframe ta med personal_id,namn,roll,avdelning,datum,vecka,dag,tid_start,tid_slut,aktivitet,typ,instruktör,anteckningar

    Hittar du inget så returnerar du bara ett exempel som du hittar på.
    """,
    tools = [get_kliniskt_schema]
)

print('hämta personalschema Agent created successfully!')

hämta personalschema Agent created successfully!


In [12]:
hamta_personalschema_agent_card = AgentCard(
    name='hämta personalschema',
    url='http://localhost:10023',
    description='Hämta schemat för en medarbetare',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='hamta_schema_personal',
            name='hämta schema personal',
            description='Hämtar schema för en medarbetare',
            tags = ['schema'],
            examples=[
                "när jobbar Emma Andersson?",
            ],
        )
    ],
)

In [13]:
remote_skapa_personalschema = RemoteA2aAgent(
    name='hamta_schema_personal',
    description='Hämta schema för en medarbetare',
    agent_card=f'http://localhost:10023{AGENT_CARD_WELL_KNOWN_PATH}',
)

/tmp/ipykernel_23227/4226252900.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_skapa_personalschema = RemoteA2aAgent(


Agent - Schemaläggare

In [14]:
skapa_schema_agent = Agent(
    model=azure_model,
    name='skapa_schema',
    instruction="""
    Du är en agent som lägger schemat för utbildningar.

    Du har tillgång till följande input:
    - Kurser som metarbetaren ska gå
    - Schema för när kurserna ges
    - Medarbetarens schema

    Regler: 
    - Medarbetare kan endast gå kurser när de är schemalagda som Fri arbetstid
    - Medarbetare ska inte anmälas till kurser som går när de är lediga
    - Medarbetaren måste vara schemalagd hela tiden som kursen pågår
    - Endast en kurs får planeras vid en given tid, inag överlapp.
    - Om det inte finns en passande tid, boka inte in kursen utan tala om att det inte finns en tid som passar.
    
    Returnera en lista med kurser som medarbetaren ska anmälas till som passar dess schema och 
    en lista med kurser som inte gick att boka in. 
    """
)

print('skapa_schema Agent created successfully!')

skapa_schema_agent_card = AgentCard(
    name='skapa kursschema',
    url='http://localhost:10024',
    description='Skapa kursschema för en medarbetare',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='skapa_schema_personal',
            name='skapa schema personal',
            description='Skapar kursschema för en medarbetare',
            tags = ['schema'],
            examples=[
                "När kan Emma Andersson gå på kurs?",
            ],
        )
    ],
)

remote_skapa_kursschema = RemoteA2aAgent(
    name='skapa_kursschema',
    description='Skapa kursschema för en medarbetare',
    agent_card=f'http://localhost:10024{AGENT_CARD_WELL_KNOWN_PATH}',
)

skapa_schema Agent created successfully!


/tmp/ipykernel_23227/2218435.py:48: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_skapa_kursschema = RemoteA2aAgent(


### Agent 3: Host Agent (Orchestrator)

The Host Agent coordinates between the other two agents to provide comprehensive trend analysis.

In [15]:
# Create the Host ADK Agent
host_agent = SequentialAgent(
    name='utbildnings_host',
    sub_agents=[remote_hamta_utbildningar, remote_skapa_utbildningsschema, remote_skapa_personalschema, remote_skapa_kursschema],
)

In [16]:
host_agent_card = AgentCard(
    name='utbildnings schema läggare host',
    url='http://localhost:10022',
    description='Orchestrates, sequentially, skapande utav schema baserat på vad som medarbetaren förväntas ta för kurser och vad det finns för utbud samt vilken tillgänglighet som personen har',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='utbildnings_schema_läggare',
            name='utbildnings schema läggare',
            description='Orchestrates, sequentially, skapande utav schema baserat på vad som medarbetaren förväntas ta för kurser och vad det finns för utbud samt vilken tillgänglighet som personen har',
            tags = ['utbildning'],
            examples=[
                'när ska sjuksköterska gå sina utbildningar?',
                "Skapa ett schema för sjuksköterskan",
            ],
        )
    ],
)

## 3. Running

Now let's put everything together. We'll create helper functions to start our agents and run the complete system.

### Starting the A2A Servers

Create function to run each agent as an A2A server:

In [17]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [18]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    # Create tasks for all servers
    tasks = [
        asyncio.create_task(
            run_agent_server(hamta_utbildningar_agent, hamta_utbildningar_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(skapa_utbildning_agent, skapa_utbildning_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(hamta_personalschema_agent, hamta_personalschema_agent_card, 10023)
        ),
        asyncio.create_task(
            run_agent_server(skapa_schema_agent, skapa_schema_agent_card, 10024)
        ),
        asyncio.create_task(
            run_agent_server(host_agent, host_agent_card, 10022)
        ),
    ]

    # Give servers time to start
    await asyncio.sleep(2)

    print('✅ All agent servers started!')
    print('   - Hämta utbildningar Agent: http://127.0.0.1:10020')
    print('   - Skapa Utbildningsutbud Agent: http://127.0.0.1:10021')
    print('   - Hämta personalschema Agent: http://127.0.0.1:10023')
    print('   - Skapa kursschema Agent: http://127.0.0.1:10024')
    print('   - Host Agent: http://127.0.0.1:10022')

    # Keep servers running
    try:
        await asyncio.gather(*tasks)
    except KeyboardInterrupt:
        print('Shutting down servers...')


# Run in a background thread


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Start the thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

# Wait for servers to be ready
time.sleep(3)

/tmp/ipykernel_23227/2220589873.py:19: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  config = A2aAgentExecutorConfig()
/tmp/ipykernel_23227/2220589873.py:20: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  executor = A2aAgentExecutor(runner=runner, config=config)


✅ All agent servers started!
   - Hämta utbildningar Agent: http://127.0.0.1:10020
   - Skapa Utbildningsutbud Agent: http://127.0.0.1:10021
   - Hämta personalschema Agent: http://127.0.0.1:10023
   - Skapa kursschema Agent: http://127.0.0.1:10024
   - Host Agent: http://127.0.0.1:10022


## 4. Testing the System

### Call the A2A agents (the 2 remote agents, and the host agent that refers to the 2 remote agents as sub agents)

In [19]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [20]:
a2a_client = A2ASimpleClient()

In [ ]:
async def test_hamta_utbildningar() -> None:
    """Test hämta utbiöldningar agent."""
    ssk_utbildningar = await a2a_client.create_task(
        'http://localhost:10020', "Vilka utbildningar ska en sjuksköterska ta?"
    )
    print(ssk_utbildningar)


# Run the async function
asyncio.run(test_hamta_utbildningar())

/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/executor/a2a_agent_executor.py:202: UserWarning: [EXPERIMENTAL] convert_a2a_request_to_agent_run_request: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  run_request = self._config.request_converter(
/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/converters/request_converter.py:115: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will

{
  "sjuksköterska": [
    {
      "kurs_id": "01",
      "kurs_namn": "Vuxen HLR med defibrillator (AED)",
      "kategori": "HLR",
      "beskrivning": "Grundkurs i hjärt-lungräddning för vuxna inklusive användning av automatisk extern defibrillator.",
      "obligatorisk": "ja"
    },
    {
      "kurs_id": "02",
      "kurs_namn": "Brandsäkerhet och släckövning",
      "kategori": "brandutbildning",
      "beskrivning": "Teori och praktiska övningar i brandförebyggande åtgärder och användning av handbrandsläckare.",
      "obligatorisk": "ja"
    },
    {
      "kurs_id": "03",
      "kurs_namn": "Basal hygien och smittskydd",
      "kategori": "infektionskontroll",
      "beskrivning": "Riktlinjer för handhygien, skyddsutrustning, isoleringsrutiner och smittspårning.",
      "obligatorisk": "ja"
    },
    {
      "kurs_id": "04",
      "kurs_namn": "Läkemedelshantering och säker ordination",
      "kategori": "läkemedel",
      "beskrivning": "Rutiner för säker ordination, admini

/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/executor/a2a_agent_executor.py:238: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_events: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  for a2a_event in self._config.event_converter(
/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/converters/event_converter.py:564: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_message: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. You

In [ ]:
async def test_utbildningsschema() -> None:
    """Test utbildningsschema agent."""
    utbildningsschema = await a2a_client.create_task(
        'http://localhost:10021', 'När går brandutbildningen?'
    )
    print(utbildningsschema)


# Run the async function
asyncio.run(test_utbildningsschema())

/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/executor/a2a_agent_executor.py:238: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_events: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  for a2a_event in self._config.event_converter(
/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/converters/event_converter.py:564: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_message: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. You

Vilken variant av brandutbildning menar du? (t.ex. grundläggande brandskydd, repetitionskurs, praktisk släckövning eller e‑learning) — och för vilken grupp/avdelning gäller det?

Svara kort så listar jag de närmaste tillfällena och markerar eventuella överlappande kurstillfällen.


In [23]:
async def test_host_analysis() -> None:
    """Test host analysis agent."""
    host_analysis = await a2a_client.create_task(
        'http://localhost:10022',
        'Jag heter Emma Andersson och är sjuksköterska. Kan du skapa ett utbildningschema till mig?',
    )
    print(host_analysis)


# Run the async function
asyncio.run(test_host_analysis())

/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/executor/a2a_agent_executor.py:202: UserWarning: [EXPERIMENTAL] convert_a2a_request_to_agent_run_request: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  run_request = self._config.request_converter(
/mnt/c/git/wp3-hackathon/.venv/lib/python3.11/site-packages/google/adk/a2a/converters/request_converter.py:115: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will

Tack Emma — jag har skapat ett förslag utifrån de föreslagna tillfällena och de justeringar som gjordes för att undvika krockar. Jag antog att de tider som användes är markerade som Fri arbetstid i ditt schema. Nedan är listan över kurser jag kan anmäla dig till (utan överlapp) och en lista över kurser som inte gick att boka.

Kurser som jag kan anmäla dig till:
1. Kurs_id 01 — Grundläggande patientöversättning  
   Datum: 2025-01-06, 09:00–11:00  
   Plats: Sal A  
   Instruktör: Anna Svensson  
   Notering: Obligatorisk

2. Kurs_id 03 — Infektionskontroll och hygien  
   Datum: 2025-01-07, 10:00–14:00  
   Plats: Laboratorie 1  
   Instruktör: Maria Johansson  
   Notering: Obligatorisk (flyttad för att undvika krock)

3. Kurs_id 02 — Arbetsmiljö och säkerhet  
   Datum: 2025-01-08, 13:00–16:00  
   Plats: Sal C  
   Instruktör: Per Andersson  
   Notering: Obligatorisk (flyttad för att undvika krock)

4. Kurs_id 04 — Electronic Health Records (EHR)  
   Datum: 2025-01-13, 09:00–14:0

## Summary

Congratulations! You've successfully built a multi-agent system using Google's A2A protocol. Here's what you've learned:

1. **A2A Protocol Basics**: How agents discover and communicate with each other
2. **ADK Integration**: Creating ADK agents and wrapping them for A2A
3. **Agent Orchestration**: Building a Host Agent that coordinates multiple agents
4. **Practical Implementation**: Running and testing a complete multi-agent system

### Next Steps

- **Deploy Your Agents**: Deploy agents to Cloud Run or other platforms
- **Add Authentication**: Implement security for production use
- **Create More Agents**: Build agents for your specific use cases, even using other frameworks
- **Advanced Patterns**: Explore agent chains, parallel execution, and more
- **Callbacks**: Add in the Google ADK agents the before and after callbacks of the agent, model and tool, to increase observability

Happy agent building! 🚀

# Appendix

### Why Use Google A2A (Agent-to-Agent) Protocol

Google's Agent-to-Agent (A2A) protocol is a standardized communication framework that enables AI agents to discover, communicate, and collaborate with each other using a common JSON-RPC based protocol.  
It provides a uniform way for agents to interact, regardless of their underlying implementation.  

#### 1. Standardized Communication Protocol

- A2A provides a consistent, JSON-RPC based protocol that any agent can implement
- Agents can communicate without needing to know each other's internal implementation details
- The protocol supports streaming responses for real-time interactions

#### 2. Agent Discovery and Metadata

- Agents expose their capabilities through standardized metadata (AgentCard)
- Each agent publishes its skills, input/output modes, and capabilities
- Host agents can dynamically discover what other agents can do through the `.well-known/agent-card.json` endpoint

#### 3. Orchestration and Composition

- Enables building complex multi-agent systems where a host agent can orchestrate multiple specialized agents
- Supports sequential and parallel task execution patterns
- Allows for sophisticated agent collaboration workflows

#### 4. Platform Independence

- A2A servers can wrap agents from different frameworks (not just ADK)
- Agents can be deployed as independent services on different infrastructure
- Promotes loose coupling between agents

### Differences: Using ADK Agents Directly vs. Through A2A

#### Using ADK Agents Directly

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.5-pro")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.5-pro",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)
```

__Use Direct ADK for Multi-Agents System When:__

- All agents are tightly related and always used together
- Google ADK is the framework choice, and simplicity is prioritized
- Performance of in-process communication is critical
- You don't need distributed deployment
- No built-in service discovery is needed

#### Using ADK Agents Through A2A

__Use A2A for Multi-Agents System When:__

- Building complex multi-agent systems
- Agents need to be developed, deployed, and scaled independently
- You want to integrate agents from different teams or frameworks
- You need dynamic agent discovery and composition
- Building a platform where agents can be added/removed dynamically
- You want to enable third-party agent integration